In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from colorcloud.biasutti2019riu import LitModel, LitDataModule
import lightning as L
import wandb
from lightning.pytorch.loggers import WandbLogger
from datetime import datetime
from torch import nn

In [ ]:
def wandb_hook(logger, step):
    def hook(module, input, output):
        if isinstance(module, nn.Conv2d):
            acts = output.detach()
            min_count = (acts < 1e-1).sum()
            total_count = acts.numel()
            logger.log({"min_act_rate/" + str(module.name): min_count/total_count}, step=step)
    return hook

In [ ]:
model = LitModel(debugging=True, debugging_hook=wandb_hook)
data = LitDataModule(train_batch_size=10)

In [ ]:
time = datetime.now()
timestamp = str(time.year) + '-' + str(time.month) + '-' + str(time.day) + '_' + str(time.hour) + '-' + str(time.minute) + '-' + str(time.second)

wandb_logger = WandbLogger(project="colorcloud", name=timestamp, log_model="all")
wandb_logger.watch(model.net, log="all")

In [ ]:
# train model
trainer = L.Trainer(max_epochs=2, logger=wandb_logger)
trainer.fit(model, data)

In [ ]:
wandb.finish()